# Recolección de datos y análisis descriptivo de variables 

Para nuestra investigacion acerca de la relacion del fenomeno del niño & niña con respecto al mercado electrico , se procede a descargar
las variables de interes las cuales serian : 

- costos marginales historicos 
- niveles de embalses 
- precipitaciones historicas en zonas de interés (zonas centro y sur donde se ubican generalmente los embalses de regulación por el coordinador)

Respecto al forecasting de adaptacion del modelo seran : 

- Actualización NOAA 
- Pronostico precipitacion 
- Pronostico de Cotas 

Para esto se hara descarga de las variables segun las fuentes

In [1]:
import os 
import pandas as pd 
import plotly.express as px
import folium

# Embalses localizados geograficamente 

In [2]:
df = pd.read_csv('lagos_202312191947.csv')

# Crear un mapa base
mapa = folium.Map(location=[df['latitud'].mean(), df['longitud'].mean()], zoom_start=5)

# Agregar marcadores para cada lago
for idx, row in df.iterrows():
    folium.Marker(
        location=[row['latitud'], row['longitud']],
        popup=f"{row['nombre_real']}",
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
#mapa.save('mapa_lagos.html')


In [3]:
mapa

como se obervan los embalses estan ubicados en las zona centro y sur del pais , zonas en las que nos enfocaremos principalmente en las 
zonas de valor de 
- costo marginal
- lluvias 
- generación de las centrales alimentadas por los elmbalses directamente 
- afluentes de embalses 
- cotas de embalses 

Se realizara la exploracion de datos desde que año hay historico para empezar con el modelo

# Cotas embalses , afluentes y precipitaciones 

https://www.coordinador.cl/operacion/graficos/operacion-real/cotas-y-niveles-de-embalses-reales/

### Cotas Embalses :

Los distintos embalses presentan datos desde el 2015 pero cada embalse fue implementando sensores por la DGA , logrando la comunicación 
de los embalses en tiempo real para el monitoreo y teniendo mejor control ante una desregulación no controlada. 

Para presentar los datos de embalses desde el 2016 hasta fines del 2023 es que exploraremos los distintos niveles de embalses.

El archivo que se descarga de la pagina del coordinador es de tipo tcv , pero para el traspaso y correcta manipulación de los datos los transformamos a csv. 

In [4]:
df_cotas = pd.read_csv("cotas_embalses.csv")
df_cotas

,id,fecha,altura_msnm,nombre,id_embalse
0,33643,2014-12-31 00:00:00.000,0.00,dcolbu,1
1,33654,2014-12-31 00:00:00.000,735.63,dpolcu,9
2,33648,2014-12-31 00:00:00.000,1316.09,dinver,6
3,33642,2014-12-31 00:00:00.000,102.60,dpuyehue,14
4,33652,2014-12-31 00:00:00.000,508.19,dpangu,11
...,...,...,...,...,...
45951,62615,2023-12-26 00:00:00.000,1318.07,dinver,6
45952,62614,2023-12-26 00:00:00.000,104.66,drapel,5
45953,62613,2023-12-26 00:00:00.000,316.43,dangos,4
45954,62612,2023-12-26 00:00:00.000,239.41,dchapo,3


In [5]:
# Filtrar los registros donde altura_msnm es 0
df_altura_cero = df_cotas[df_cotas['altura_msnm'] == 0]

# Análisis estadístico básico de la columna altura_msnm excluyendo los valores 0
df_sin_ceros = df_cotas[df_cotas['altura_msnm'] > 0]
analisis_estadistico = df_sin_ceros['altura_msnm'].describe()

# Conteo de registros con altura 0
conteo_ceros = df_altura_cero.shape[0]

conteo_ceros, analisis_estadistico

(932,
 count    45024.000000
 mean       649.788854
 std        568.972933
 min         25.190000
 25%        231.560000
 50%        507.490000
 75%        735.172500
 max       2163.990000
 Name: altura_msnm, dtype: float64)

Tras un detallado análisis de los datos recopilados en el conjunto de datos cotas_embalses.csv, se han identificado ciertas anomalías que requieren atención. Una de las más notorias es la presencia de valores cero en la columna altura_msnm, que representa la altura sobre el nivel del mar de diversos embalses. Considerando la naturaleza de estos datos, es altamente improbable que la altura real sobre el nivel del mar de un embalse sea cero. Por lo tanto, estos valores son indicativos de errores en los registros o de períodos en los cuales las estaciones de medición no estaban funcionando adecuadamente.

Una observación crucial que surge de nuestro análisis es que, a partir de un punto específico en el tiempo (presumiblemente desde el año 2016), los datos muestran una mayor consistencia y fiabilidad. Esto sugiere que las estaciones de medición comenzaron a operar de manera más efectiva o que los procesos de recopilación de datos experimentaron mejoras significativas. Este cambio en la calidad de los datos es un indicador positivo de que las mediciones actuales son más representativas y precisas.

Para asegurar la integridad y la calidad del análisis basado en este conjunto de datos, es esencial abordar estos valores anómalos. La limpieza de los datos debería implicar la eliminación o corrección de los registros que contienen valores de altura de cero. Dicha acción permitirá obtener análisis más precisos y conclusiones fiables, evitando así distorsiones que puedan surgir de datos erróneos o incompletos. Este paso es crucial para mantener la confiabilidad y la exactitud en la interpretación de los datos de los embalses.

In [6]:
nombres_unicos = df_cotas['nombre'].unique()
id_embalses_unicos = df_cotas['id_embalse'].unique()


print("Nombres únicos:", nombres_unicos)
print("IDs de embalses únicos:", id_embalses_unicos)

Nombres únicos: ['dcolbu' 'dpolcu' 'dinver' 'dpuyehue' 'dpangu' 'dmelad' 'dlaja' 'dmaule'
 'drapel' 'dangos' 'dchapo' 'dmachi' 'dralco' 'dpullin']
IDs de embalses únicos: [ 1  9  6 14 11 13  8  7  5  4  3  2 10 12]


Hay 14 embalses pero los embalses de nombre dpuyehue y dpullin son las cotas de los lagos puyehue y pullinque , siendo estas las que 
alimentan las centrales Pullinque y Pilmaiquen pero estas son de tipo Pasada , por lo que seran filtradas de este dataset.

Segun las centrales de embalse que son reguladas por el coordinador : 
Documentos de programacion de embalses y centrales : 

https://www.coordinador.cl/operacion/documentos/programas-de-operacion-2021/

Los embalses son los siguientes : 

- Lago Chapo 
- La invernada 
- Lago Laja 
- Colbún 
- Rapel
- Ralco 
- Melado 
- Maule 
- Pangue 
- Polcura 
- Angostura 

In [7]:
# Convertir la columna 'fecha' a datetime
df_cotas['fecha'] = pd.to_datetime(df_cotas['fecha'])

# Obtener los IDs únicos de embalses
id_embalses_unicos = df_cotas['id_embalse'].unique()

# Diccionario para almacenar cada DataFrame
dfs = {}

for id_embalse in id_embalses_unicos:
    # Filtrar por id_embalse y ordenar por fecha
    df_filtrado = df_cotas[df_cotas['id_embalse'] == id_embalse].sort_values(by='fecha')
    # Almacenar el DataFrame en el diccionario
    dfs[id_embalse] = df_filtrado

In [8]:
for id_embalse in id_embalses_unicos:
    # Crear una variable de DataFrame para cada id_embalse
    exec(f"df_embalse_{id_embalse} = dfs[{id_embalse}]")

### Embalse colbun 

In [9]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_1['fecha'] = pd.to_datetime(df_embalse_1['fecha'])

# Filtrar para mantener solo los valores donde altura_msnm es mayor a 0
df_embalse_1_filtrado = df_embalse_1[df_embalse_1['altura_msnm'] > 0]

# Crear un gráfico de área para la serie de tiempo
fig = px.area(df_embalse_1_filtrado, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Colbún')

# Ajustar el rango del eje y para que comience cerca del valor mínimo de altura_msnm
min_altura = df_embalse_1_filtrado['altura_msnm'].min()
fig.update_yaxes(range=[min_altura, df_embalse_1_filtrado['altura_msnm'].max()])

# Mostrar el gráfico
fig.show()

primera medicion : 21 - 08 - 2015 

### Embalse Chapo

In [23]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_3['fecha'] = pd.to_datetime(df_embalse_3['fecha'])

# Filtrar para mantener solo los valores donde altura_msnm es mayor a 0
df_embalse_3 = df_embalse_3[df_embalse_3['altura_msnm'] > 0]

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_3, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Chapo')

# Mostrar el gráfico
fig.show()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_14160\904749319.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



primera medicion : 21 - 08 - 2015 

### Embalse Angostura

In [11]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_4['fecha'] = pd.to_datetime(df_embalse_4['fecha'])

# Filtrar para mantener solo los valores donde altura_msnm es mayor a 0
df_embalse_4 = df_embalse_4[df_embalse_4['altura_msnm'] > 0]

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_4, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Angostura')

# Mostrar el gráfico
fig.show()

### Embalse Rapel

In [12]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_5['fecha'] = pd.to_datetime(df_embalse_5['fecha'])

# Filtrar para mantener solo los valores donde altura_msnm es mayor a 0
df_embalse_5 = df_embalse_5[df_embalse_5['altura_msnm'] > 0]

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_5, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Rapel')

# Mostrar el gráfico
fig.show()

### Embalse Invernada

In [13]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_6['fecha'] = pd.to_datetime(df_embalse_6['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_6, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Invernada')

# Mostrar el gráfico
fig.show()

### Embalse Maule

In [14]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_7['fecha'] = pd.to_datetime(df_embalse_7['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_7, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Maule')

# Mostrar el gráfico
fig.show()

### Embalse Laja

In [15]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_8['fecha'] = pd.to_datetime(df_embalse_8['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_8, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Laja')

# Mostrar el gráfico
fig.show()

### Embalse Polcura

In [16]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_9['fecha'] = pd.to_datetime(df_embalse_9['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_9, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Polcura')

# Mostrar el gráfico
fig.show()

### Embalse Ralco

In [17]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_10['fecha'] = pd.to_datetime(df_embalse_10['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_10, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Ralco')

# Mostrar el gráfico
fig.show()

### Embalse Pangue

In [18]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_11['fecha'] = pd.to_datetime(df_embalse_11['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_11, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Pangue')

# Mostrar el gráfico
fig.show()

### Embalse Melado

In [19]:
# Asegúrate de que 'fecha' es una columna de tipo datetime
df_embalse_13['fecha'] = pd.to_datetime(df_embalse_13['fecha'])

# Crear un gráfico de línea para la serie de tiempo
fig = px.line(df_embalse_13, x='fecha', y='altura_msnm', title='Serie de Tiempo de Altura (msnm) para el Embalse Melado')

# Mostrar el gráfico
fig.show()

In [22]:
import requests

def consultar_mediciones(fecha_inicio, fecha_fin, id_estacion, id_variable):
    # URL de la API y credenciales proporcionadas
    url = "https://snia.mop.gob.cl/extractor-rest/mediciones"
    correo = "ignacio.molina@enel.com"
    password = "g!S8I7h7"

    # Datos para la solicitud
    datos = {
        "correo": correo,
        "password": password,
        "fecha_inicio": fecha_inicio,
        "fecha_fin": fecha_fin,
        "id_estacion": id_estacion,
        "id_variable": id_variable
    }

    # Realizar la solicitud POST
    response = requests.post(url, json=datos)

    # Verificar si la solicitud fue exitosa y retornar el resultado
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

# Ejemplo de uso de la función
fecha_inicio = "20231201 00:00:00"
fecha_fin = "20231201 01:00:00"
id_estacion = "07321007-0"
id_variable = "SCBD0200"

# Llamada a la función
# resultado = consultar_mediciones(fecha_inicio, fecha_fin, id_estacion, id_variable)
# print(resultado)
resultado = consultar_mediciones(fecha_inicio, fecha_fin, id_estacion, id_variable)
print(resultado)


[{'estacion': '07321007-0', 'fecha_medicion': '2023-12-01 00:18:00', 'medicion_calc': '2.786', 'medicion_real': '2.696', 'caudal': '21.971'}]


In [21]:
# Llamada a la función
resultado = consultar_mediciones(fecha_inicio, fecha_fin, id_estacion, id_variable)
print(resultado)

KeyboardInterrupt: 

In [ ]:
import requests
from datetime import datetime, timedelta

def consultar_mediciones_con_limites(fecha_inicio, horas=60):
    """
    Realiza consultas a la API cada hora desde la fecha de inicio proporcionada.
    Continúa hasta que se alcance el límite de horas o hasta que se encuentre un error.
    Devuelve el número de consultas exitosas y la respuesta de la última consulta.
    """
    url = "https://snia.mop.gob.cl/extractor-rest/mediciones"
    correo = "ignacio.molina@enel.com"
    password = "g!S8I7h7"
    id_estacion = "10322003-3"
    id_variable = "SCBD0200"

    fecha_actual = datetime.strptime(fecha_inicio, "%Y%m%d %H:%M:%S")
    contador = 0
    ultima_respuesta = None

    for _ in range(horas):
        # Convertir la fecha en formato requerido por la API
        fecha_fin = fecha_actual + timedelta(hours=1)
        fecha_inicio_str = fecha_actual.strftime("%Y%m%d %H:%M:%S")
        fecha_fin_str = fecha_fin.strftime("%Y%m%d %H:%M:%S")

        # Preparar los datos de la solicitud
        datos = {
            "correo": correo,
            "password": password,
            "fecha_inicio": fecha_inicio_str,
            "fecha_fin": fecha_fin_str,
            "id_estacion": id_estacion,
            "id_variable": id_variable
        }

        # Realizar la solicitud
        response = requests.post(url, json=datos)

        # Si la solicitud es exitosa, aumentar el contador y guardar la respuesta
        if response.status_code == 200:
            contador += 1
            ultima_respuesta = response.json()
        else:
            break

        print(ultima_respuesta)
        # Preparar la siguiente fecha de inicio
        fecha_actual += timedelta(hours=1)

    return contador, ultima_respuesta

# Ejemplo de uso de la función
fecha_inicio = "20230913 14:00:00"
contador, ultima_respuesta = consultar_mediciones_con_limites(fecha_inicio)
print(f"Total de consultas exitosas: {contador}")
print("Última respuesta:", ultima_respuesta)



[{'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 14:08:00', 'medicion_calc': '0.89', 'medicion_real': '0.89', 'caudal': '47.967'}, {'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 14:38:00', 'medicion_calc': '0.89', 'medicion_real': '0.89', 'caudal': '47.967'}]
[{'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 15:08:00', 'medicion_calc': '0.89', 'medicion_real': '0.89', 'caudal': '47.967'}, {'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 15:38:00', 'medicion_calc': '0.9', 'medicion_real': '0.9', 'caudal': '48.774'}]
[{'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 16:08:00', 'medicion_calc': '0.9', 'medicion_real': '0.9', 'caudal': '48.774'}, {'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 16:38:00', 'medicion_calc': '0.89', 'medicion_real': '0.89', 'caudal': '47.967'}]
[{'estacion': '10322003-3', 'fecha_medicion': '2023-09-13 17:08:00', 'medicion_calc': '0.89', 'medicion_real': '0.89', 'caudal': '47.967'}, {'estacion': '10322003-3', 

In [ ]:
import requests

def consultar_precipitacion_acumulada(fecha_inicio, fecha_fin, id_estacion):
    url = "https://snia.mop.gob.cl/extractor-rest/mediciones"
    correo = "ignacio.molina@enel.com"
    password = "g!S8I7h7"
    id_variable = "SCBD0221"  # Variable para la precipitación acumulada anual

    # Realizar la solicitud para la fecha de inicio
    datos_inicio = {
        "correo": correo,
        "password": password,
        "fecha_inicio": fecha_inicio,
        "fecha_fin": fecha_fin,  # Misma hora que fecha_inicio
        "id_estacion": id_estacion,
        "id_variable": id_variable
    }
    respuesta = requests.post(url, json=datos_inicio)

    if respuesta.status_code == 200:
        return respuesta.json()
    else:
        return f"Error en la consulta: {respuesta.status_code}"

# Ejemplo de uso de la función
fecha_inicio = "20231201 00:00:00"
fecha_fin = "20231201 01:00:00"
id_estacion = "07301002-0"

# Llamada a la función
resultado = consultar_precipitacion_acumulada(fecha_inicio, fecha_fin, id_estacion)
print("Resultado de la consulta:", resultado)



Resultado de la consulta: []


In [ ]:
import requests
import pandas as pd

def consultar_mediciones(fecha_inicio, fecha_fin, id_estacion, id_variable):
    url = "https://snia.mop.gob.cl/extractor-rest/mediciones"
    correo = "ignacio.molina@enel.com"
    password = "g!S8I7h7"
    datos = {
        "correo": correo,
        "password": password,
        "fecha_inicio": fecha_inicio,
        "fecha_fin": fecha_fin,
        "id_estacion": id_estacion,
        "id_variable": id_variable
    }
    response = requests.post(url, json=datos)
    if response.status_code == 200:
        return response.json()
    else:
        return f"Error: {response.status_code}"

def leer_excel_y_consultar(fecha_inicio, fecha_fin, id_variable):
    archivo_excel = "C:\\Users\\Lenovo\\Desktop\\Soporte evolutivo\\archivos_excel\\tags_dga.xlsx"
    df = pd.read_excel(archivo_excel, sheet_name='caudales')
    for estacion in df['estacion']:
        print(f"Consultando estación: {estacion}")
        resultado = consultar_mediciones(fecha_inicio, fecha_fin, estacion, id_variable)
        print(resultado)

# Ejemplo de uso de la función
fecha_inicio = "20231201 00:00:00"
fecha_fin = "20231201 01:00:00"
id_variable = "SCBD0200"

# Llamar a la función
leer_excel_y_consultar(fecha_inicio, fecha_fin, id_variable)


Consultando estación: 06003001-4
[]
Consultando estación: 06008005-4
[{'estacion': '06008005-4', 'fecha_medicion': '2023-12-01 00:27:00', 'medicion_calc': '3.14', 'medicion_real': '3.14', 'caudal': '110.596'}, {'estacion': '06008005-4', 'fecha_medicion': '2023-12-01 00:57:00', 'medicion_calc': '3.21', 'medicion_real': '3.21', 'caudal': '124.218'}]
Consultando estación: 06008009-7
[{'estacion': '06008009-7', 'fecha_medicion': '2023-12-01 00:00:00', 'medicion_calc': '5.22', 'medicion_real': '5.22', 'caudal': '68.908'}, {'estacion': '06008009-7', 'fecha_medicion': '2023-12-01 00:30:00', 'medicion_calc': '5.26', 'medicion_real': '5.26', 'caudal': '69.64'}, {'estacion': '06008009-7', 'fecha_medicion': '2023-12-01 01:00:00', 'medicion_calc': '5.25', 'medicion_real': '5.25', 'caudal': '69.457'}]
Consultando estación: 06019003-8
[{'estacion': '06019003-8', 'fecha_medicion': '2023-12-01 00:28:00', 'medicion_calc': '11.02', 'medicion_real': '11.02', 'caudal': '126.897'}, {'estacion': '06019003-8